# (Pre)Procesado de los datos

En este cuaderno se llevará a cabo la tarea de preprocesado/procesado de los datos que se van a utilizar para el proyecto de visualización. Esta tarea consiste en una serie de subtareas, que se describen brevemente a continuación:

- **Limpieza de datos**: También conocida como _data cleaning_ o _data scrubbing_, consiste en corregir o eliminar los datos erróneos en un dataset.
- **Filtrado/selección de variables y/o instancias**: Proceso de selección para un análisis concreto del subconjunto de datos de interés.
- **Transformación de variables**: Cambio de los valores de una variable o la generación de nuevas variables a partir de variables ya existentes

Se puede acceder a los diferentes apartados de este cuaderno a través del siguiente índice:

- [Censo de animales domésticos](#Censo-de-animales-domésticos)
- [Indicadores demográficos de distritos y barrios de la ciudad de Madrid](#Demográficos)
- [Superficie de parques y zonas verdes](#Superficie-de-parques-y-zonas-verdes)
- [Áreas caninas en Madrid](#Áreas-caninas-en-Madrid)

Después del trabajo realizado en este cuaderno, contamos con una serie de archivos .csv con los datos limpios, filtrados y seleccionados listos para su correlación y visualización.

Para acceder al cuaderno con la descripción de los datos, [hacer click aquí](./Entrega_intermedia_1.ipynb).

## Censo de animales domésticos

Este primer conjunto de datos será sencillo de preprocesar, dado que los datos en crudo (en el .csv proporcionado por el Ayuntamiento de Madrid) ya se encuentran bien estructurados y sin valores erróneos.

### Limpieza de datos

Como se ha visto en el cuaderno _Descripción de los conjuntos de datos_, este _dataset_ no presenta valores nulos ni datos erróneos en general, por lo que no será necesario llevar a cabo una limpieza de datos.

### Filtrado/selección de variables y/o instancias

Como ya se ha indicado anteriormente, este conjunto de datos contiene información desde el año 2014 hasta el año 2023. Dado que vamos a trabajar únicamente con información desde el 2019 al 2023, podemos prescindir de aquellas instancias correspondienes a años anteriores al 2018:

In [1]:
#Importamos la biblioteca pandas
import pandas as pd

#Cargamos la información del .csv en el dataframe con el que vamos a trabajar
censo = pd.read_csv('datasets/censo animales 2014-2023.csv', delimiter=';')

#Filtramos los años que no nos interesan
censo = censo[(censo['AÑO'] >= 2019) & (censo['AÑO'] <= 2023)]

#Comprobamos que ya no salen las entradas correspondientes a los años previos al 2019
print(censo)

      AÑO           DISTRITO  ESPECIE CANINA  ESPECIE FELINA
0    2023         ARGANZUELA           12685            7028
1    2023            BARAJAS            6068            2176
2    2023        CARABANCHEL           24302            9955
3    2023             CENTRO           19791           12457
4    2023          CHAMARTÍN           13632            5181
..    ...                ...             ...             ...
100  2019             TETUÁN           12470            5535
101  2019              USERA           12393            2898
102  2019          VICÁLVARO            5244            1505
103  2019  VILLA DE VALLECAS            9923            2946
104  2019         VILLAVERDE           12917            2694

[105 rows x 4 columns]


### Transformación de variables

Considerando que el resto de conjuntos de datos con los que se va a trabajar disponen no sólo de la varaible **distrito** sino también de una variable adicional con el **código del distrito**, sería interesante incorporar a este dataframe una columna nueva con la numeración correspondiente a cada distrito. De esta manera, cuando más adelante se quiera correlacionar los diferentes conjuntos de datos, se podrá hacer a través de los códigos de los distritos, salvando posibles discrepancias que pueda haber en la redacción de los nombres de los diferentes distritos (uso de mayúsculas, abreviaciones, guiones, etc).


In [2]:
# Creamos un diccionario con el código correspondiente a cada distrito
distrito_codigo = {
    'CENTRO': 1,
    'ARGANZUELA': 2,
    'RETIRO': 3,
    'SALAMANCA': 4,
    'CHAMARTÍN': 5,
    'TETUÁN': 6,
    'CHAMBERÍ': 7,
    'FUENCARRAL-EL PARDO': 8,
    'MONCLOA-ARAVACA': 9,
    'LATINA': 10,
    'CARABANCHEL': 11,
    'USERA': 12,
    'PUENTE DE VALLECAS': 13,
    'MORATALAZ': 14,
    'CIUDAD LINEAL': 15,
    'HORTALEZA': 16,
    'VILLAVERDE': 17,
    'VILLA DE VALLECAS': 18,
    'VICÁLVARO': 19,
    'SAN BLAS-CANILLEJAS': 20,
    'BARAJAS': 21
}

# Añadimos la columna 'CÓDIGO DISTRITO' al dataframe
censo.insert(1, 'cod_distrito', censo['DISTRITO'].map(distrito_codigo))

print(censo)

      AÑO  cod_distrito           DISTRITO  ESPECIE CANINA  ESPECIE FELINA
0    2023             2         ARGANZUELA           12685            7028
1    2023            21            BARAJAS            6068            2176
2    2023            11        CARABANCHEL           24302            9955
3    2023             1             CENTRO           19791           12457
4    2023             5          CHAMARTÍN           13632            5181
..    ...           ...                ...             ...             ...
100  2019             6             TETUÁN           12470            5535
101  2019            12              USERA           12393            2898
102  2019            19          VICÁLVARO            5244            1505
103  2019            18  VILLA DE VALLECAS            9923            2946
104  2019            17         VILLAVERDE           12917            2694

[105 rows x 5 columns]


Una información adicional que puede ser interesante para el proyecto de visualización de datos es transformar los datos de este _dataset_ para tener información nueva, como por ejemplo el número total de animales domésticos por distrito, el número total de animales domésticos en la ciudad de Madrid cada año, etc. Además, podemos añadir una nueva columna al conjunto de datos original con el total de animales domésticos por distrito.

In [3]:
# Nueva columna en el dataframe original con el total de perros + gatos en cada distrito en cada año
censo['TOTAL'] = censo['ESPECIE CANINA'] + censo['ESPECIE FELINA']

# Ordenamos las entradas por código de distrito y por año
censo = censo.sort_values(by=['cod_distrito', 'AÑO'], ascending=[True, True])
# Reseteamos los índices
censo = censo.reset_index(drop=True)
# Cambiamos "año" por "year"
censo.rename(columns={'AÑO': 'YEAR'}, inplace=True)

# Exportamos a .csv
censo.to_csv('preprocesado/censo.csv',sep=';', index = False)

# Nuevo dataframe
# Total de perros por año
total_perros = censo.groupby('YEAR')['ESPECIE CANINA'].sum().reset_index()
# Total de gatos por año
total_gatos = censo.groupby('YEAR')['ESPECIE FELINA'].sum().reset_index()
# Total de perros + gatos por año
total_animales = censo.groupby('YEAR')['TOTAL'].sum().reset_index()

# Creamos el dataframe con los datos anuales
censo_anual = total_perros.copy()
censo_anual['ESPECIE FELINA'] = total_gatos['ESPECIE FELINA']
censo_anual['TOTAL'] = total_animales['TOTAL']
censo_anual = censo_anual.sort_values(by=['YEAR'], ascending=[True])

# Reseteamos los índices
censo_anual = censo_anual.reset_index(drop=True)

# Exportamos a .csv
censo_anual.to_csv('preprocesado/censo_anual.csv',sep=';', index = False)

In [4]:
# Comprobamos
censo.head()

,YEAR,cod_distrito,DISTRITO,ESPECIE CANINA,ESPECIE FELINA,TOTAL
0,2019,1,CENTRO,16010,9248,25258
1,2020,1,CENTRO,16202,9477,25679
2,2021,1,CENTRO,16442,9878,26320
3,2022,1,CENTRO,16943,10645,27588
4,2023,1,CENTRO,19791,12457,32248


Comprobamos si el nuevo dataframe con los valores totales de animales domésticos por año se ha creado correctamente:

In [5]:
# Comprobamos
censo_anual.head()

,YEAR,ESPECIE CANINA,ESPECIE FELINA,TOTAL
0,2019,276873,105683,382556
1,2020,280002,108750,388752
2,2021,282315,113991,396306
3,2022,283935,126522,410457
4,2023,334189,151517,485706


Llegados a este punto, podemos considerar que el conjunto de datos del censo de animales domésticos ya está listo para ser procesado y correlado con el resto de conjuntos de datos.

<a id="Demográficos"></a> 

## Indicadores demográficos de distritos y barrios de la ciudad de Madrid

A diferencia del _dataset_ anterior, este sí que presenta una serie de desafíos a abordar en la fase de preprocesado debido a su ingente cantidad de datos y el carácter ambiguo de sus categorías. Iremos de nuevo paso a paso limpiando y filtrando los datos del conjunto para su posterior procesado.

### Limpieza de datos

Aunque hemos visto en la **descripción de los datos** que este _dataset_ contiene valores nulos, estos se corresponden con celdas vacías en las categorías del código de distrito o barrio (porque hay entradas que abarcan toda la ciudad), así como para ciertas categorías o subcategorías. Por eso, en primer lugar llevaremos a cabo el filtrado de los datos y posteriormente volveremos a la fase de limpieza por si en el subconjunto de datos de interés aún quedan valores erróneos o nulos que debamos eliminar.

In [10]:
# Cargamos la información del .csv en el dataframe con el que vamos a trabajar
demo_distritos = pd.read_csv('datasets/indicadores distritos 2019-2023.csv', delimiter=';', encoding='ISO-8859-1')

# Comprobamos
demo_distritos.head()

,Orden,Periodo panel,ciudad,cod_distrito,distrito,cod_barrio,barrio,año,fecha_indicador,fuente_indicador,categoría_1,categoría_2,indicador_nivel1,indicador_nivel2,indicador_nivel3,unidad_indicador,indicador_completo,valor_indicador
0,1,2019,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Características Generales del Distrito-Barrio,NaN,Superficie,NaN,NaN,Ha,Superficie (Ha.),60.446
1,2,2019,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Características Generales del Distrito-Barrio,NaN,Población densidad de población,NaN,NaN,Habitantes/Ha.,Población densidad (hab./Ha.),54
2,3,2019,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,NaN,NaN,Habitantes,Número Habitantes,3.266.126
3,4,2019,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,Hombres,NaN,Habitantes,Población Hombres,1.521.178
4,5,2019,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,Mujeres,NaN,Habitantes,Población Mujeres,1.744.948


### Filtrado/selección de variables y/o instancias

Esta es la parte más compleja del procesado de este conjunto de datos, ya que habrá que seleccionar las categorías y/o subcategorías que nos interesan para el proyecto, eliminando las que no, para tener un _dataset_ más manejable.

Los datos filtrado o eliminados se van comentando a lo largo del código, pero principalmente se trata de quitar información que o bien sea irrelevante para los objetivos del proyecto o redundante.

In [11]:
# Eliminamos las entradas donde 'cod_barrio' o 'barrio' no son nulos
demo_distritos = demo_distritos[demo_distritos['cod_barrio'].isna() & demo_distritos['barrio'].isna()]

# Eliminamos las columnas 'cod_barrio' y 'barrio'
demo_distritos = demo_distritos.drop(columns=['cod_barrio', 'barrio'])

# Eliminamos las entradas donde las variables cod_distrito y distrito sean nulas
demo_distritos = demo_distritos.dropna(subset=['distrito', 'cod_distrito'])

# Eliminamos las columnas Orden, Ciudad, Fecha del indicador, Fuente del indicador, Año (usaremos el año del panel)
demo_distritos = demo_distritos.drop(columns=['Orden', 'ciudad','fecha_indicador','fuente_indicador','año'])

# Cambiamos el nombre de la columna Periodo panel a Año para simplificar
demo_distritos = demo_distritos.rename(columns={'Periodo panel': 'YEAR'})

# Convertimos 'cod_distrito' a entero
demo_distritos['cod_distrito'] = demo_distritos['cod_distrito'].astype(int)

# Separamos los datos de la variable 2020-21
df_2020_21 = demo_distritos[demo_distritos['YEAR'] == '2020-21'].copy()
# Creamos dos copias de este dataframe para 2020 y 2021
df_2020 = df_2020_21.copy()
df_2021 = df_2020_21.copy()
# Cambiamos el valor de la columna Año
df_2020['YEAR'] = 2020
df_2021['YEAR'] = 2021

demo_distritos = pd.concat([demo_distritos[demo_distritos['YEAR'] != '2020-21'], df_2020, df_2021])

demo_distritos['YEAR'] = demo_distritos['YEAR'].astype(int)

# Reseteamos los índices
demo_distritos = demo_distritos.reset_index(drop=True)

# Comprobamos
demo_distritos.head()

,YEAR,cod_distrito,distrito,categoría_1,categoría_2,indicador_nivel1,indicador_nivel2,indicador_nivel3,unidad_indicador,indicador_completo,valor_indicador
0,2019,1,Centro,Características Generales del Distrito-Barrio,NaN,Superficie,NaN,NaN,Ha,Superficie (Ha.),523
1,2019,1,Centro,Características Generales del Distrito-Barrio,NaN,Población densidad de población,NaN,NaN,Habitantes/Ha.,Población densidad (hab./Ha.),258
2,2019,1,Centro,Población del distrito,Estructura de población,Número de Habitantes,NaN,NaN,Habitantes,Número Habitantes,134.881
3,2019,1,Centro,Población del distrito,Estructura de población,Número de Habitantes,Hombres,NaN,Habitantes,Población Hombres,67.829
4,2019,1,Centro,Población del distrito,Estructura de población,Número de Habitantes,Mujeres,NaN,Habitantes,Población Mujeres,67.052


Para facilitar trabajar con estos datos, vamos a crear una serie de subconjuntos en diferentes _dataframes_ según la categoría que estemos abordando:

- Estructura de la población (Edad media por distrito, habitantes, número de hombres y mujeres, etc)
- Indicadores económicos (nivel de renta media)
- Indicadores de salud en la población (sólo se disponen datos del panel del año 2019)
- Resultados de elecciones locales
  - Aunque en 2019 Más Madrid presentó candidatura única y en 2023 lo hizo junto a Verdes Equo, vamos a unificar ambos indicadores para facilitar el trabajo
  - Lo mismo para Ciudadanos (2019) y Ciudadanos-Partido de la Ciudadanía (2023)

#### Estructura de la población
Número de habitantes total, mujeres, hombres, etc., por distrito y año

In [8]:
# Extraemos el subconjunto con la info de la población por distrito y creamos un nuevo dataframe
dist_pob = demo_distritos.loc[(demo_distritos['categoría_1'] == 'Población del distrito')].copy()
# Nos quedamos con las entradas en las que la subcategoría2 es Estructura de la población
dist_pob = dist_pob.loc[(dist_pob['categoría_2'] == 'Estructura de población')]
# Nos quedamos con las entradas en las que el indicador de nivel1 es Número de Habitantes
dist_pob = dist_pob.loc[(dist_pob['indicador_nivel1'] == 'Número de Habitantes')]
# Nos quedamos con las entradas en las que el indicador de nivel2 es Hombres, Mujeres o está vacío (hombres+mujeres)
dist_pob = dist_pob.loc[((dist_pob['indicador_nivel2'] == 'Hombres') | (dist_pob['indicador_nivel2'] == 'Mujeres') | (dist_pob['indicador_nivel2'].isnull()))]
dist_pob['indicador_nivel2'] = dist_pob['indicador_nivel2'].fillna('TOTAL')
# Nos quedamos con las entradas en las que el indicador de nivel3 es nulo y posteriormente eliminamos columnas que no nos aportan información
dist_pob = dist_pob.loc[(dist_pob['indicador_nivel3'].isnull())]
dist_pob = dist_pob.drop(columns=['indicador_nivel3', 'indicador_completo', 'unidad_indicador'])
# Formateamos los datos de la columna valor_indicador
dist_pob['valor_indicador'] = dist_pob['valor_indicador'].str.replace('.', '')
dist_pob['valor_indicador'] = dist_pob['valor_indicador'].str.replace(',', '.')
dist_pob['valor_indicador'] = dist_pob['valor_indicador'].astype(float)
# Eliminamos las columnas que no nos aportan información significativa
dist_pob = dist_pob.drop(columns=['categoría_1','categoría_2'])

# Ordenamos las entradas por código de distrito y por año
dist_pob = dist_pob.sort_values(by=['cod_distrito', 'YEAR'], ascending=[True, True])

# Reseteamos los índices
dist_pob = dist_pob.reset_index(drop=True)

# Exportamos a .csv
dist_pob.to_csv('preprocesado/dist_pob.csv',sep=';',index = False)

In [9]:
# Comprobamos
dist_pob.head()

,YEAR,cod_distrito,distrito,indicador_nivel1,indicador_nivel2,valor_indicador
0,2019,1,Centro,Número de Habitantes,TOTAL,134881.0
1,2019,1,Centro,Número de Habitantes,Hombres,67829.0
2,2019,1,Centro,Número de Habitantes,Mujeres,67052.0
3,2020,1,Centro,Número de Habitantes,TOTAL,142033.0
4,2020,1,Centro,Número de Habitantes,Hombres,72294.0


#### Indicadores económicos
Renta neta anual por distrito y año

In [10]:
# Extraemos el subconjunto con la info de los indicadores económicos y creamos un nuevo dataframe
dist_eco = demo_distritos.loc[(demo_distritos['categoría_1'] == 'Indicadores Económicos')].copy()
# Eliminamos las columnas que no nos interesan
dist_eco = dist_eco.drop(columns=['categoría_1','categoría_2','indicador_nivel3','unidad_indicador','indicador_completo'])
# Corregimos typo
dist_eco['indicador_nivel1'] = dist_eco['indicador_nivel1'].str.replace('Renta ', 'Renta')
# Seguimos filtrando las entradas que no nos interesan
dist_eco = dist_eco.loc[(dist_eco['indicador_nivel1'] == 'Renta')]
dist_eco = dist_eco.loc[(dist_eco['indicador_nivel2'] == 'Neta anual')]
# Formateamos los datos de la columna valor_indicador
dist_eco['valor_indicador'] = dist_eco['valor_indicador'].str.replace('.', '')
dist_eco['valor_indicador'] = dist_eco['valor_indicador'].str.replace(',', '.')
dist_eco['valor_indicador'] = dist_eco['valor_indicador'].astype(float)

# Ordenamos las entradas por código de distrito y por año
dist_eco = dist_eco.sort_values(by=['cod_distrito','YEAR'], ascending=[True, True])

# Reseteamos los índices
dist_eco = dist_eco.reset_index(drop=True)

# Exportamos a .csv
dist_eco.to_csv('preprocesado/dist_eco.csv',sep=';',index=False)

In [11]:
# Comprobamos
dist_eco.head()

,YEAR,cod_distrito,distrito,indicador_nivel1,indicador_nivel2,valor_indicador
0,2019,1,Centro,Renta,Neta anual,32458.0
1,2020,1,Centro,Renta,Neta anual,33473.0
2,2021,1,Centro,Renta,Neta anual,33473.0
3,2022,1,Centro,Renta,Neta anual,36072.0
4,2023,1,Centro,Renta,Neta anual,36984.0


#### Indicadores de salud
Porcentaje de sedentarismo entre la población de cada distrito (sólo hay datos del panel de 2019)

In [12]:
# Extraemos el subconjunto con la info de indicadores de salud por distrito y creamos un nuevo dataframe
dist_salud = demo_distritos.loc[(demo_distritos['categoría_1'] == 'Salud')].copy()
# Corregimos typo
dist_salud['categoría_2'] = dist_salud['categoría_2'].str.replace('Hábitos y estilos de vida ', 'Hábitos y estilos de vida')
# Filtramos las entradas que no nos interesan
dist_salud = dist_salud.loc[(dist_salud['categoría_2'] == 'Hábitos y estilos de vida')]
dist_salud = dist_salud.loc[(dist_salud['indicador_nivel2'] == 'Sedentarismo')]
# Eliminamos las columnas que no nos interesan
dist_salud = dist_salud.drop(columns=['categoría_1','indicador_nivel1', 'indicador_nivel3','unidad_indicador','indicador_completo'])
# Formateamos los datos de la columna valor_indicador
dist_salud['valor_indicador'] = dist_salud['valor_indicador'].str.replace(',', '.')
dist_salud['valor_indicador'] = dist_salud['valor_indicador'].astype(float)*100
dist_salud['valor_indicador'] = dist_salud['valor_indicador'].round(2)
# Ordenamos las entradas por código de distrito y por año
dist_salud = dist_salud.sort_values(by=['cod_distrito', 'YEAR'], ascending=[True, True])
# Reseteamos los índices
dist_salud = dist_salud.reset_index(drop=True)

# Exportamos a .csv
dist_salud.to_csv('preprocesado/dist_salud.csv',sep=';',index=False)

In [13]:
# Comprobamos
dist_salud.head()

,YEAR,cod_distrito,distrito,categoría_2,indicador_nivel2,valor_indicador
0,2019,1,Centro,Hábitos y estilos de vida,Sedentarismo,27.0
1,2019,2,Arganzuela,Hábitos y estilos de vida,Sedentarismo,26.0
2,2019,3,Retiro,Hábitos y estilos de vida,Sedentarismo,24.0
3,2019,4,Salamanca,Hábitos y estilos de vida,Sedentarismo,31.0
4,2019,5,Chamartín,Hábitos y estilos de vida,Sedentarismo,25.0


#### Elecciones locales 2019 y 2023
Porcentaje de votos a cada partido por distrito en las elecciones locales del año 2019 y año 2023 respecto al censo total (incluyendo abstenciones y votos en blanco)

In [12]:
# Extraemos el subconjunto con la info de las elecciones locales por distrito y creamos un nuevo dataframe
dist_elec = demo_distritos.loc[(demo_distritos['categoría_1'] == 'Resultados Elecciones Locales')].copy()
# Eliminamos columnas vacías y/o que no nos aportan información
dist_elec = dist_elec.drop(columns=['categoría_2','indicador_nivel3','indicador_completo','unidad_indicador'])
# Corregimos typo
dist_elec['indicador_nivel2'] = dist_elec['indicador_nivel2'].str.replace('VOX ', 'VOX')
# Unificamos entradas de las elecciones 2019 y 2023
dist_elec['indicador_nivel2'] = dist_elec['indicador_nivel2'].str.replace('Más Madrid-Verdes Equo', 'Más Madrid')
dist_elec['indicador_nivel2'] = dist_elec['indicador_nivel2'].str.replace('Ciudadanos-Partido de la Ciudadanía', 'Ciudadanos')
# Formateamos los datos de la columna valor_indicador
dist_elec['valor_indicador'] = dist_elec['valor_indicador'].str.replace('.', '')
# Eliminamos entradas que no nos interesan
dist_elec = dist_elec.loc[(dist_elec['indicador_nivel2'] != 'Votos a candidaturas') & (dist_elec['indicador_nivel2'] != 'Abstenciones en elecciones municipales') & (dist_elec['indicador_nivel2'] != 'Votos en blanco')]
# Ordenamos las entradas por código de distrito y por año
dist_elec = dist_elec.sort_values(by=['cod_distrito', 'YEAR'], ascending=[True, True])
# Reseteamos los índices
dist_elec = dist_elec.reset_index(drop=True)

# Exportamos a .csv
dist_elec.to_csv('preprocesado/dist_elec.csv',sep=';',index = False)

In [15]:
# Comprobamos
dist_elec.head(10)

,YEAR,cod_distrito,distrito,categoría_1,indicador_nivel1,indicador_nivel2,valor_indicador
0,2019,1,Centro,Resultados Elecciones Locales,Número de votos,Censo,98873
1,2019,1,Centro,Resultados Elecciones Locales,Número de votos,Más Madrid,32421
2,2019,1,Centro,Resultados Elecciones Locales,Número de votos,Partido Popular,11379
3,2019,1,Centro,Resultados Elecciones Locales,Número de votos,Ciudadanos,8851
4,2019,1,Centro,Resultados Elecciones Locales,Número de votos,Partido Socialista Obrero Español,6069
5,2019,1,Centro,Resultados Elecciones Locales,Número de votos,VOX,3625
6,2023,1,Centro,Resultados Elecciones Locales,Número de votos,Censo,95771
7,2023,1,Centro,Resultados Elecciones Locales,Número de votos,Ciudadanos,1790
8,2023,1,Centro,Resultados Elecciones Locales,Número de votos,Más Madrid,20281
9,2023,1,Centro,Resultados Elecciones Locales,Número de votos,Partido Popular,20456


### Transformación de variables

Centrándonos en el último _dataframe_ que hemos creado con la información de los votos en las elecciones municipales de 2019 y 2020 por distrito madrileño, resultaría más interesante trabajar directamente con porcentajes en lugar de con valores crudos de número de votos, dadas las diferencias sustanciales en cuanto a la población (y por tanto, censo electoral) entre distritos. Por ello, vamos a volver a cargar el dataframe _dist_elec_ y vamos a añadir una nueva columna con estos datos porcentuales.

In [13]:
del dist_elec_anual

# Función para calcular el porcentaje de cada partido sobre el total del censo
def calcular_porcentaje(grupo):
    total = grupo[grupo['indicador_nivel2'] == 'Censo']['valor_indicador'].astype(float).values[0]
    grupo['porcentaje'] = 100 * grupo['valor_indicador'].astype(float) / total
    return grupo

# Hacemos una copia para conservar el dataframe original
dist_elec_anual = dist_elec.copy()
dist_elec_anual = dist_elec_anual.groupby(['YEAR', 'cod_distrito', 'distrito'])[['YEAR', 'cod_distrito', 'distrito', 'valor_indicador', 'indicador_nivel2']].apply(calcular_porcentaje).reset_index(drop=True)

# Redondeamos los porcentajes a dos decimales
dist_elec_anual['porcentaje'] = dist_elec_anual['porcentaje'].round(2)
# El número de votos en bruto no es de utilidad
dist_elec_anual = dist_elec_anual.drop(columns=['valor_indicador'])
# Ordenamos las entradas por código de distrito y por año
dist_elec_anual = dist_elec_anual.sort_values(by=['cod_distrito', 'YEAR'], ascending=[True, True])
# Reseteamos los índices
dist_elec_anual = dist_elec_anual.reset_index(drop=True)

# Exportamos a .csv
dist_elec_anual.to_csv('preprocesado/dist_elec_anual.csv',sep=';',index = False)

In [18]:
# Comprobamos
dist_elec_anual.head(10)

,YEAR,cod_distrito,distrito,indicador_nivel2,porcentaje
0,2019,1,Centro,Censo,100.00
1,2019,1,Centro,Más Madrid,32.79
2,2019,1,Centro,Partido Popular,11.51
3,2019,1,Centro,Ciudadanos,8.95
4,2019,1,Centro,Partido Socialista Obrero Español,6.14
5,2019,1,Centro,VOX,3.67
6,2023,1,Centro,Censo,100.00
7,2023,1,Centro,Ciudadanos,1.87
8,2023,1,Centro,Más Madrid,21.18
9,2023,1,Centro,Partido Popular,21.36


## Superficie de parques y zonas verdes

Tal y como se vio en el cuaderno con la descripción de las variables, este se trata de un conjunto de datos describe las superficies de parques y zonas verdes por distrito de Madrid desde el año 2017 hasta el año 2023. Dato que el Portal de Datos Abiertos del Ayuntamiento de Madrid proporciona estos datos en archivos .csv separados para cada año, lo primero que se va a hacer dentro de la tarea de procesado es unir los datos de los años de interés (es decir, de 2019 al 2023).

In [1]:
import pandas as pd

#Cargamos la información de todos los .csv con los que vamos a trabajar
arbolado_19 = pd.read_csv('datasets/arbolado-2019.csv', delimiter=';', encoding='ISO-8859-1')
arbolado_19.insert(0, 'año', 2019)
arbolado_20 = pd.read_csv('datasets/arbolado-2020.csv', delimiter=';', encoding='ISO-8859-1')
arbolado_20.insert(0, 'año', 2020)
arbolado_21 = pd.read_csv('datasets/arbolado-2021.csv', delimiter=';', encoding='ISO-8859-1')
arbolado_21.insert(0, 'año', 2021)
arbolado_22 = pd.read_csv('datasets/arbolado-2022.csv', delimiter=';', encoding='ISO-8859-1')
arbolado_22.insert(0, 'año', 2022)
arbolado_23 = pd.read_csv('datasets/arbolado-2023.csv', delimiter=';', encoding='ISO-8859-1')
arbolado_23.insert(0, 'año', 2023)

### Limpieza de datos

Además de unir todos los .csv en un único _dataframe_, también será necesario llevar a cabo la limpieza de datos de valores erróneos o nulos, columnas repetidas, errores tipográficos, etc, tal y como se he hecho anteriormente en los otros conjuntos de datos.

In [2]:
arbolado_total = [arbolado_19, arbolado_20, arbolado_21, arbolado_22, arbolado_23]
arbolado = pd.concat(arbolado_total)

# Eliminamos columnas que no nos interesan
arbolado = arbolado.drop(columns=['Nº DISTRITO','PARQUES Y ZONAS VERDES SIGNIFICATIVAS DE DISTRITO','PARQUES Y ZONAS VERDES MÁS SIGNIFICATIVAS DE DISTRITO',
                                  'Superficie (m2)','m2 de ZONAS VERDES Y PARQUES en Distrito 2022', 'm2 de ZONAS VERDES Y PARQUES en distrito',
                                  'PARQUES Y ZONAS VERDES DE DISTRITO CON MAYOR SUPERFICIE','Nº Distrito','BARRIO','ï»¿NÂº Distrito'])

# Formateamos la columna DISTRITO y corregimos typos
arbolado['DISTRITO'] = arbolado['DISTRITO'].replace({
    'CHAMARTIN': 'CHAMARTÍN',
    'CHAMARTÃN': 'CHAMARTÍN',
    'TETUÃN': 'TETUÁN',
    'CHAMBERÃ': 'CHAMBERÍ',
    'FUENCARRAL - EL PARDO': 'FUENCARRAL-EL PARDO',
    'SAN BLAS - CANILLEJAS': 'SAN BLAS-CANILLEJAS',
    'TETUAN': 'TETUÁN',
    'CHAMBERI': 'CHAMBERÍ',
    'VICALVARO': 'VICÁLVARO',
    'VICÃLVARO': 'VICÁLVARO',
    'MONCLOA - ARAVACA': 'MONCLOA-ARAVACA'
})

# Creamos un diccionario con el código correspondiente a cada distrito
distrito_codigo = {
    'CENTRO': 1,
    'ARGANZUELA': 2,
    'RETIRO': 3,
    'SALAMANCA': 4,
    'CHAMARTÍN': 5,
    'TETUÁN': 6,
    'CHAMBERÍ': 7,
    'FUENCARRAL-EL PARDO': 8,
    'MONCLOA-ARAVACA': 9,
    'LATINA': 10,
    'CARABANCHEL': 11,
    'USERA': 12,
    'PUENTE DE VALLECAS': 13,
    'MORATALAZ': 14,
    'CIUDAD LINEAL': 15,
    'HORTALEZA': 16,
    'VILLAVERDE': 17,
    'VILLA DE VALLECAS': 18,
    'VICÁLVARO': 19,
    'SAN BLAS-CANILLEJAS': 20,
    'BARAJAS': 21
}

# Añadimos la columna 'cod_distrito' al dataframe
arbolado.insert(1, 'cod_distrito', arbolado['DISTRITO'].map(distrito_codigo))

### Filtrado/selección de variables y/o instancias

En este apartado se filtrarán las columnas y/o entradas que no nos interesan para el proyecto de visualización de datos, ya sea por redundancia o por contener información que no resulta relevante.

In [3]:
# Eliminamos filas que no nos interesan o que están vacías
arbolado = arbolado[arbolado['DISTRITO'] != 'Total:']
columnas = arbolado.columns.tolist()
columnas.remove('año')
arbolado = arbolado.dropna(subset=columnas, how='all')

# Unificamos las columnas de superficie
arbolado['SUPERFICIE (ha)'] = arbolado['SUPERFICIE (ha)'].fillna(arbolado['Superficie (ha)'])
arbolado['SUPERFICIE (ha)'] = arbolado['SUPERFICIE (ha)'].fillna(arbolado['ha de ZONAS VERDES Y PARQUES en Distrito 2022'])
arbolado['SUPERFICIE (ha)'] = arbolado['SUPERFICIE (ha)'].fillna(arbolado['ha de ZONAS VERDES Y PARQUES en distrito '])

# Eliminamos las columnas originales que ya no necesitamos
arbolado = arbolado.drop(['Superficie (ha)', 'ha de ZONAS VERDES Y PARQUES en Distrito 2022', 'ha de ZONAS VERDES Y PARQUES en distrito '], axis=1)

# Formateamos cod_distrito
arbolado['cod_distrito'] = arbolado['cod_distrito'].astype(int)

# Formateamos los datos de la columna SUPERFICIE
arbolado['SUPERFICIE (ha)'] = arbolado['SUPERFICIE (ha)'].str.replace(',', '.')
arbolado['SUPERFICIE (ha)'] = arbolado['SUPERFICIE (ha)'].astype(float)

### Transformación de variables

Por último, vamos a crear un nuevo dataframe en el que se agrupen los valores de superficie de las zonas verdes por distrito y año, resultando en un subconjunto de datos mucho más fácil de manegar e inteligible.

In [7]:
# Agrupamos por 'año' y 'DISTRITO' y sumamos los valores de 'SUPERFICIE (ha)'
arbolado_anual = arbolado.groupby(['año', 'DISTRITO'])['SUPERFICIE (ha)'].sum().reset_index()
arbolado_anual['SUPERFICIE (ha)'] = arbolado_anual['SUPERFICIE (ha)'].round(2)
arbolado_anual.rename(columns={'año': 'YEAR'}, inplace=True)
# Exportamos a .csv
arbolado_anual.to_csv('preprocesado/arbolado_anual.csv',sep=';', index = False)

In [9]:
# Comprobamos
arbolado_anual.head()

## Áreas caninas en Madrid

Por último, vamos a trabajar con este conjunto de datos en el que se indican las zonas caninas disponibles a lo largo de la ciudad de Madrid. En esta ocasión vuelve a suceder como con los datos de los parques y zonas verdes, que la información está separada en diferentes .csv en cada año. Por suerte, podremos abordarlo de manera análoga al caso anterior.

### Limpieza de datos

Unificamos todos los .csv en un único _dataframe_ de _pandas_ y limpiamos valores erróneos o nulos.

In [8]:
#Cargamos la información del .csv en el dataframe con el que vamos a trabajar
caninas_19 = pd.read_csv('datasets/areas caninas-2019.csv', delimiter=';', encoding='ISO-8859-1')
caninas_19.insert(0, 'año', 2019)
caninas_20 = pd.read_csv('datasets/areas caninas-2020.csv', delimiter=';', encoding='ISO-8859-1')
caninas_20.insert(0, 'año', 2020)
caninas_21 = pd.read_csv('datasets/areas caninas-2021.csv', delimiter=';', encoding='ISO-8859-1')
caninas_21.insert(0, 'año', 2021)
caninas_22 = pd.read_csv('datasets/areas caninas-2022.csv', delimiter=';', encoding='ISO-8859-1')
caninas_22.insert(0, 'año', 2022)
caninas_23 = pd.read_csv('datasets/areas caninas-2023.csv', delimiter=';', encoding='ISO-8859-1')
caninas_23.insert(0, 'año', 2023)

In [9]:
caninas_total = [caninas_19, caninas_20, caninas_21, caninas_22, caninas_23]
caninas = pd.concat(caninas_total)

caninas.head()

,año,"MXASSETNUM,C,12","DESCRIPTIO,C,105","MINTDISTRI,C,20","NOMBRE_DIS,C,25","MINTBARRIO,C,10","NOMBRE_BAR,C,50","INSTALLDAT,D","MINTTIPOVI,C,15","MINTNOMBRE,C,254","MINTNUMERO,C,25","MINTNDP,C,20","TIPO,C,254","TOTAL_ELEM,N,10,0","ESTADO,C,50","ï»¿MXASSETNUM,C,12","NOMBRE_DIS,C,100","NOMBRE_BAR,C,100"
0,2019,3580435.0,C2073-Canina-AV ARCENTALES 29,20,SAN BLAS-CANILLEJAS,201,SIMANCAS,01/01/04,AVENIDA,ARCENTALES,29,20128779.0,CANINAS,4,ACTIVO,NaN,NaN,NaN
1,2019,3580438.0,C2076-Canina-C ARCOS DE JALON 16,20,SAN BLAS-CANILLEJAS,204,ARCOS,01/01/04,CALLE,ARCOS DEL JALON,16,11135550.0,CANINAS,2,ACTIVO,NaN,NaN,NaN
2,2019,3568735.0,C1057-Canina,10,LATINA,101,LOS CARMENES,03/10/16,NaN,NaN,NaN,NaN,CANINAS DE JUEGOS,8,ACTIVO,NaN,NaN,NaN
3,2019,3580440.0,C2078-Canina-C ARCOS DE JALON 13,20,SAN BLAS-CANILLEJAS,204,ARCOS,01/05/16,CALLE,ARCOS DEL JALON,13,31032028.0,CANINAS,1,ACTIVO,NaN,NaN,NaN
4,2019,16829.0,C0350-Canina,3,RETIRO,32,ADELFAS,01/07/14,CALLE,ARREGUI Y ARUEJ,16,31005668.0,CANINAS DE JUEGOS,4,ACTIVO,NaN,NaN,NaN


### Filtrado/selección de variables y/o instancias

A continuación vamos a filtrar el conjunto de datos para quedarnos únicamente con el subconjunto que nos interesa. Para ello, al igual que en apartados anteriores, eliminaremos información irrelevante o redundante.

In [10]:
# Eliminamos columnas que no nos interesan
caninas = caninas.drop(columns=['MXASSETNUM,C,12','MINTBARRIO,C,10','NOMBRE_BAR,C,50','INSTALLDAT,D','MINTTIPOVI,C,15','MINTNOMBRE,C,254','MINTNUMERO,C,25',
                                 'MINTNDP,C,20','ï»¿MXASSETNUM,C,12','NOMBRE_BAR,C,100','TIPO,C,254','TOTAL_ELEM,N,10,0'])
# Unificamos las columnas de DISTRITO
caninas['NOMBRE_DIS,C,25'] = caninas['NOMBRE_DIS,C,25'].fillna(caninas['NOMBRE_DIS,C,100'])

# Nos quedamos sólo con las zonas caninas en activo
caninas = caninas.loc[(caninas['ESTADO,C,50'] == 'ACTIVO')]

# Eliminamos columnas redundantes
caninas = caninas.drop(columns=['NOMBRE_DIS,C,100','ESTADO,C,50'])

# Renombramos las columnas para que sean coherentes con el resto de datasets
caninas = caninas.rename(columns={'MINTDISTRI,C,20': 'cod_distrito', 'NOMBRE_DIS,C,25': 'distrito'})

In [11]:
# Comprobamos
caninas.head(10)

,año,"DESCRIPTIO,C,105",cod_distrito,distrito
0,2019,C2073-Canina-AV ARCENTALES 29,20,SAN BLAS-CANILLEJAS
1,2019,C2076-Canina-C ARCOS DE JALON 16,20,SAN BLAS-CANILLEJAS
2,2019,C1057-Canina,10,LATINA
3,2019,C2078-Canina-C ARCOS DE JALON 13,20,SAN BLAS-CANILLEJAS
4,2019,C0350-Canina,3,RETIRO
5,2019,C0604 - Canina - C/ MARTIRES DE PARACUELLOS Nº2,6,TETUAN
6,2019,C2171-Canina-C BANDE 14,21,BARAJAS
7,2019,C2172-Canina-AV LOGROÑO 98,21,BARAJAS
9,2019,C1572-Canina-C CANAL DE PANAMA 17,15,CIUDAD LINEAL
10,2019,C1573-Canina-C VIRGEN DE LLUC 44,15,CIUDAD LINEAL


### Transformación de variables

Al igual que en el caso anterior, resultaría de utilidad contar con un _dataset_ reducido y simplificado con el recuento de áreas caninas por cada distrito y año.

In [12]:
# Nuevo dataframe con el recuento de áreas caninas por distrito y año
caninas_anual = caninas.groupby(['año', 'cod_distrito', 'distrito']).size().reset_index(name='TOTAL')
caninas_anual.rename(columns={'año' : 'YEAR'}, inplace=True)
# Exportamos a .csv
caninas_anual.to_csv('preprocesado/caninas_anual.csv',sep=';', index = False)

In [13]:
#Comprobamos
caninas_anual.head()

,YEAR,cod_distrito,distrito,TOTAL
0,2019,1,CENTRO,2
1,2019,2,ARGANZUELA,6
2,2019,3,RETIRO,10
3,2019,4,SALAMANCA,5
4,2019,5,CHAMARTIN,1
